In [4]:
import getpass
import nibabel as nib
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import os

In [5]:
user = getpass.getuser()

nifti_dir = "/home/ORUNET/" + user + "/2303_Chalmers/Data/BWT/Nifti"
nifti_paths = list(Path(nifti_dir).glob('**/*.nii'))

seg_dir = "/home/ORUNET/" + user + "/2303_Chalmers/Data/BWT/Segmentations"
r_mask_paths = list(Path(seg_dir).glob('**/mask_R.nii.gz'))
l_mask_paths = list(Path(seg_dir).glob('**/mask_L.nii.gz'))

dst_dir = "/home/ORUNET/" + user + "/2303_Chalmers/Data/BWT/Preprocessed"

In [6]:
def main():
    for nifti, r_mask, l_mask in zip(nifti_paths, r_mask_paths, l_mask_paths):
        print(nifti)

#main()

/home/ORUNET/hanta231/2303_Chalmers/Data/BWT/Nifti/1001_bwt.nii
/home/ORUNET/hanta231/2303_Chalmers/Data/BWT/Nifti/1002_bwt.nii
/home/ORUNET/hanta231/2303_Chalmers/Data/BWT/Nifti/1003_bwt.nii
/home/ORUNET/hanta231/2303_Chalmers/Data/BWT/Nifti/1004_bwt.nii
/home/ORUNET/hanta231/2303_Chalmers/Data/BWT/Nifti/1005_bwt.nii
/home/ORUNET/hanta231/2303_Chalmers/Data/BWT/Nifti/1006_bwt.nii
/home/ORUNET/hanta231/2303_Chalmers/Data/BWT/Nifti/1007_bwt.nii
/home/ORUNET/hanta231/2303_Chalmers/Data/BWT/Nifti/1008_bwt.nii
/home/ORUNET/hanta231/2303_Chalmers/Data/BWT/Nifti/1009_bwt.nii
/home/ORUNET/hanta231/2303_Chalmers/Data/BWT/Nifti/1010_bwt.nii
/home/ORUNET/hanta231/2303_Chalmers/Data/BWT/Nifti/1011_bwt.nii
/home/ORUNET/hanta231/2303_Chalmers/Data/BWT/Nifti/1012_bwt.nii
/home/ORUNET/hanta231/2303_Chalmers/Data/BWT/Nifti/1013_bwt.nii
/home/ORUNET/hanta231/2303_Chalmers/Data/BWT/Nifti/1014_bwt.nii
/home/ORUNET/hanta231/2303_Chalmers/Data/BWT/Nifti/1015_bwt.nii
/home/ORUNET/hanta231/2303_Chalmers/Data

In [ ]:
mask = nib.load(seg_dir + "/1001_bwt/mask_R.nii.gz").get_fdata()

In [ ]:
start_z = 0

for z in range(mask.shape[2]):
    if mask[:,:,z].any():
        start_z = z
        break

stop_z = mask.shape[2]

for z in reversed(range(mask.shape[2])):
    if mask[:,:,z].any():
        stop_z = z
        break

print("start:", start_z, "stop:", stop_z)

start_x = 0

for x in range(mask.shape[0]):
    if mask[x,:,:].any():
        start_x = x
        break

stop_x = mask.shape[0]

for x in reversed(range(mask.shape[0])):
    if mask[x,:,:].any():
        stop_x = x
        break

print("start:", start_x, "stop:", stop_x)




In [ ]:
nib_file = nib.load(nifti_paths[0])
img_affine = nib_file.affine
nib_imgs = nib_file.get_fdata()

In [ ]:
nib_imgs = nib_imgs[start_x:stop_x,50:-50, 200:201] 

low_cv, high_cv = -1024, 250

for i, im in enumerate(nib_imgs):
    if any(any(c > high_cv or c < low_cv for c in r) for r in im):
        nib_imgs[i] = [[max(min(c, high_cv), low_cv) for c in r] for r in im]

In [ ]:
def get_z_start_stop(mask):
    start_z = 0

    for z in range(mask.shape[2]):
        if mask[:,:,z].any():
            start_z = z
            break

    stop_z = mask.shape[2]

    for z in reversed(range(mask.shape[2])):
        if mask[:,:,z].any():
            stop_z = z
            break

    return start_z, stop_z


def get_x_start_stop(mask):
    start_x = 0

    for x in range(mask.shape[0]):
        if mask[x,:,:].any():
            start_x = x
            break

    stop_x = mask.shape[0]

    for x in reversed(range(mask.shape[0])):
        if mask[x,:,:].any():
            stop_x = x
            break

    return start_x, stop_x

def generate_chunks(mask_l, mask_r):
    
    masks = [mask_l, mask_r]

    for mask in masks:
        start_z, stop_z = get_z_start_stop(mask)
        start_x, stop_x = get_x_start_stop(mask)
    